# 机器学习第二次实验

## 理解并描述决策树分类、回归算法原理

决策树是一种**监督**学习算法，广泛应用于分类和回归任务。其核心思想是基于数据属性进行**递归**划分，构建一个树状模型。在决策树中，内部节点代表一个属性上的测试，每个分支代表测试的一个结果，叶节点代表最终的决策结果。

### 决策树分类算法原理:

1. **特征选择**：常用的特征选择标准包括信息增益（ID3），信息增益率（C4.5）和基尼指数（CART）。信息增益高的特征具有更强的分类能力，基尼指数则用于CART算法，衡量数据的不纯度，基尼指数越小，数据纯度越高。

    - **信息增益**：计算每个特征分割前后的信息熵变化，选择使信息熵降低最多的特征。
    - **信息增益率**：对信息增益结果进行归一化处理，解决信息增益偏向于选择取值较多的特征的问题。
    - **基尼指数**：CART算法使用，反映了从数据集中随机抽取两个样本，其类别标签不一致的概率。

2. **树的构建**：从根节点开始，使用特征选择方法选择最佳特征，根据该特征的不同取值构建分支。对每个分支递归执行同样的过程，直至满足停止条件，如节点中样本数小于最小分割样本数、节点纯度达到阈值或达到预设的最大深度。

3. **剪枝**：构建完成后，通过剪枝来避免过拟合。剪枝分为预剪枝和后剪枝，预剪枝是在构建树的过程中提前停止树的增长，后剪枝则是先构建完整的树，然后删除掉一些子树或节点。

### 决策树回归算法原理：

1. **特征选择**：通常基于最小化均方误差（MSE）或总方差减少来选择最佳分割特征和分割点。

2. **树的构建**：选择最佳分割特征后，按此特征的值分割数据集，生成两个子节点，并对每个子节点递归重复此过程，直到满足停止条件。

3. **预测**：对于回归树的叶节点，其预测值通常是到达该叶节点的所有样本目标值的均值。

## 决策树分类与回归算法设计

In [1]:
# 决策树分类器

import numpy as np

class DecisionTreeNode:
    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

def calculate_entropy(y):
    _, counts = np.unique(y, return_counts=True)
    probabilities = counts / counts.sum()
    entropy = -np.sum(probabilities * np.log2(probabilities))
    return entropy

def split_dataset(X, y, feature, threshold):
    left_indices = X[:, feature] < threshold
    right_indices = ~left_indices
    return X[left_indices], X[right_indices], y[left_indices], y[right_indices]

def best_split(X, y):
    best_feature, best_threshold, best_gain = None, None, float("-inf")
    base_entropy = calculate_entropy(y)
    for feature in range(X.shape[1]):
        thresholds = np.unique(X[:, feature])
        for threshold in thresholds:
            _, _, y_left, y_right = split_dataset(X, y, feature, threshold)
            if len(y_left) == 0 or len(y_right) == 0:
                continue
            entropy_left, entropy_right = calculate_entropy(y_left), calculate_entropy(y_right)
            weighted_entropy = (len(y_left) / len(y)) * entropy_left + (len(y_right) / len(y)) * entropy_right
            information_gain = base_entropy - weighted_entropy
            if information_gain > best_gain:
                best_feature, best_threshold, best_gain = feature, threshold, information_gain
    return best_feature, best_threshold

def build_tree(X, y, depth=0, max_depth=None):
    if len(np.unique(y)) == 1 or (max_depth is not None and depth == max_depth):
        value = np.argmax(np.bincount(y))
        return DecisionTreeNode(value=value)
    feature, threshold = best_split(X, y)
    if feature is None:
        return DecisionTreeNode(value=np.argmax(np.bincount(y)))
    X_left, X_right, y_left, y_right = split_dataset(X, y, feature, threshold)
    left = build_tree(X_left, y_left, depth + 1, max_depth)
    right = build_tree(X_right, y_right, depth + 1, max_depth)
    return DecisionTreeNode(feature=feature, threshold=threshold, left=left, right=right)

def predict(node, X):
    if node.value is not None:
        return node.value
    if X[node.feature] < node.threshold:
        return predict(node.left, X)
    else:
        return predict(node.right, X)

def decision_tree_predict(tree, X):
    return np.array([predict(tree, x) for x in X])

In [2]:
# 决策树回归器

import numpy as np

class DecisionTreeNode:
    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

def calculate_mse(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

def split_dataset(X, y, feature, threshold):
    left_indices = X[:, feature] < threshold
    right_indices = ~left_indices
    return X[left_indices], X[right_indices], y[left_indices], y[right_indices]

def best_split(X, y):
    best_feature, best_threshold, best_mse = None, None, float("inf")
    for feature in range(X.shape[1]):
        thresholds = np.unique(X[:, feature])
        for threshold in thresholds:
            X_left, X_right, y_left, y_right = split_dataset(X, y, feature, threshold)
            if len(y_left) == 0 or len(y_right) == 0:
                continue
            mse_left = calculate_mse(y_left, np.mean(y_left))
            mse_right = calculate_mse(y_right, np.mean(y_right))
            weighted_mse = (len(y_left) / len(y)) * mse_left + (len(y_right) / len(y)) * mse_right
            if weighted_mse < best_mse:
                best_feature, best_threshold, best_mse = feature, threshold, weighted_mse
    return best_feature, best_threshold

def build_tree(X, y, depth=0, max_depth=None):
    if len(np.unique(y)) == 1 or (max_depth is not None and depth == max_depth):
        value = np.mean(y)
        return DecisionTreeNode(value=value)
    feature, threshold = best_split(X, y)
    if feature is None:
        return DecisionTreeNode(value=np.mean(y))
    X_left, X_right, y_left, y_right = split_dataset(X, y, feature, threshold)
    left = build_tree(X_left, y_left, depth + 1, max_depth)
    right = build_tree(X_right, y_right, depth + 1, max_depth)
    return DecisionTreeNode(feature=feature, threshold=threshold, left=left, right=right)

def predict(node, X):
    if node.value is not None:
        return node.value
    if X[node.feature] < node.threshold:
        return predict(node.left, X)
    else:
        return predict(node.right, X)

def decision_tree_predict(tree, X):
    return np.array([predict(tree, x) for x in X])

## 数据集获取

### 用于分类的数据集

1. 鸢尾花数据集（Iris）：包含150个样本，分为3个类别，每个类别50个样本。每个样本有4个特征，分别是花瓣和花萼的长度和宽度。
2. 葡萄酒数据集（Wine）：包含178个样本，分为3个类别，代表了三种不同的意大利葡萄酒。有13个特征，这些特征是从葡萄酒的化学成分分析中得出的，比如酒精度、苹果酸含量等。

### 用于回归的数据集

1. 波士顿房价数据集（Boston Housing）：包含506个样本和13个特征，目标是预测波士顿地区的房屋价格中位数。
2. 糖尿病数据集（Diabetes）：包含442个样本和10个基于生理特征的特征（年龄、性别、BMI等），目标是一年后病情的量化测量。

In [3]:
# 数据集获取

from sklearn.datasets import load_iris, load_wine, load_diabetes
import pandas as pd

iris = load_iris()
wine = load_wine()
diabetes = load_diabetes()

iris_df = pd.DataFrame(iris.data, columns=iris.feature_names)
iris_df['target'] = iris.target
iris_df.to_csv('iris.csv', index=False)

wine_df = pd.DataFrame(wine.data, columns=wine.feature_names)
wine_df['target'] = wine.target
wine_df.to_csv('wine.csv', index=False)

diabetes_df = pd.DataFrame(diabetes.data, columns=diabetes.feature_names)
diabetes_df['target'] = diabetes.target
diabetes_df.to_csv('diabetes.csv', index=False)

## 编程实践：将决策树算法用于具体数据集

In [4]:
# 一、使用决策树分类器对鸢尾花数据集进行分类
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier

iris_df = pd.read_csv('data/iris.csv')
print(iris_df.head())

   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
0                5.1               3.5                1.4               0.2   
1                4.9               3.0                1.4               0.2   
2                4.7               3.2                1.3               0.2   
3                4.6               3.1                1.5               0.2   
4                5.0               3.6                1.4               0.2   

   target  
0       0  
1       0  
2       0  
3       0  
4       0  


In [5]:
X, y = iris_df.iloc[:, :-1].values, iris_df.iloc[:, -1].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

tree = DecisionTreeClassifier()
tree.fit(X_train, y_train)
y_pred = tree.predict(X_test)
print(accuracy_score(y_test, y_pred))

1.0


In [6]:
# 下面想使用随机森林算法对鸢尾花数据集进行分类
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier()
forest.fit(X_train, y_train)
y_pred = forest.predict(X_test)
print(accuracy_score(y_test, y_pred))

1.0


In [7]:
# 使用网格搜索调整随机森林的参数
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [10, 20, 30, 40, 50],
    'max_depth': [3, 5, 7, 9]
}

grid_search = GridSearchCV(RandomForestClassifier(), param_grid, cv=5)
grid_search.fit(X_train, y_train)
print(grid_search.best_params_)
print(grid_search.best_score_)
y_pred = grid_search.predict(X_test)
print(accuracy_score(y_test, y_pred))

{'max_depth': 3, 'n_estimators': 30}
0.9583333333333334
1.0


### 鸢尾花数据集结论

1. 首先调用决策树分类器对鸢尾花数据集进行分类，分类准确率为 $ 100\% $
2. 然后使用随机森林算法再次进行分类，分类准确率仍为 $ 100\% $
3. 最后使用网格搜索调整随机森林的参数，得出最优参数为：最大深度 $3$，决策树个数 $30$；最优得分为 $0.958$。在此参数下，分类准确率仍为 $ 100\% $

In [8]:
# 二、使用决策树回归器对葡萄酒数据集进行分类
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier

wine_df = pd.read_csv('data/wine.csv')
print(wine_df.head())

   alcohol  malic_acid   ash  alcalinity_of_ash  magnesium  total_phenols  \
0    14.23        1.71  2.43               15.6      127.0           2.80   
1    13.20        1.78  2.14               11.2      100.0           2.65   
2    13.16        2.36  2.67               18.6      101.0           2.80   
3    14.37        1.95  2.50               16.8      113.0           3.85   
4    13.24        2.59  2.87               21.0      118.0           2.80   

   flavanoids  nonflavanoid_phenols  proanthocyanins  color_intensity   hue  \
0        3.06                  0.28             2.29             5.64  1.04   
1        2.76                  0.26             1.28             4.38  1.05   
2        3.24                  0.30             2.81             5.68  1.03   
3        3.49                  0.24             2.18             7.80  0.86   
4        2.69                  0.39             1.82             4.32  1.04   

   od280/od315_of_diluted_wines  proline  target  
0          

In [9]:
X, y = wine_df.iloc[:, :-1].values, wine_df.iloc[:, -1].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

tree = DecisionTreeClassifier()
tree.fit(X_train, y_train)
y_pred = tree.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.9444444444444444


In [10]:
# 下面想使用XGBoost对葡萄酒数据集进行分类
from xgboost import XGBClassifier

xgb = XGBClassifier()
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.9722222222222222


In [11]:
# 使用网格搜索算法进行参数优化
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [10, 30, 50, 100],
    'max_depth': [3, 6, 9]
}

grid_search = GridSearchCV(XGBClassifier(), param_grid, cv=5)
grid_search.fit(X_train, y_train)
print(grid_search.best_params_)
print(grid_search.best_score_)
y_pred = grid_search.predict(X_test)
print(accuracy_score(y_test, y_pred))

{'max_depth': 3, 'n_estimators': 30}
0.9573891625615765
0.9722222222222222


### 葡萄酒数据集结论

1. 首先调用决策树分类器对葡萄酒数据集进行分类，分类准确率为 $ 94.4\% $
2. 然后使用XGBoost算法再次进行分类，分类准确率提升至 $ 97.2\% $
3. 最后使用网格搜索调整XGB的参数，得出最优参数为：最大深度 $3$，决策树个数 $30$；最优得分为 $0.957$。在此参数下，分类准确率为 $ 97.2\% $

In [12]:
# 三、使用决策树回归器对波士顿房价数据集进行回归
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor

house_df = pd.read_csv('data/house.csv')
print(house_df.head())

      CRIM    ZN  INDUS  CHAS    NOX     RM   AGE     DIS  RAD  TAX  PTRATIO  \
0  0.00632  18.0   2.31     0  0.538  6.575  65.2  4.0900    1  296     15.3   
1  0.02731   0.0   7.07     0  0.469  6.421  78.9  4.9671    2  242     17.8   
2  0.02729   0.0   7.07     0  0.469  7.185  61.1  4.9671    2  242     17.8   
3  0.03237   0.0   2.18     0  0.458  6.998  45.8  6.0622    3  222     18.7   
4  0.06905   0.0   2.18     0  0.458  7.147  54.2  6.0622    3  222     18.7   

        B  LSTAT  MEDV  
0  396.90   4.98  24.0  
1  396.90   9.14  21.6  
2  392.83   4.03  34.7  
3  394.63   2.94  33.4  
4  396.90   5.33  36.2  


In [13]:
X, y = house_df.iloc[:, :-1].values, house_df.iloc[:, -1].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

tree = DecisionTreeRegressor()
tree.fit(X_train, y_train)
y_pred = tree.predict(X_test)
print(mean_squared_error(y_test, y_pred))

11.08343137254902


In [14]:
# 下面想使用XGBoost对波士顿房价数据集进行回归
from xgboost import XGBRegressor

xgb = XGBRegressor()
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)
print(mean_squared_error(y_test, y_pred))

6.560527271813469


In [15]:
# 使用网格搜索算法进行参数优化
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [10, 30, 50, 100],
    'max_depth': [3, 6, 9]
}

grid_search = GridSearchCV(XGBRegressor(), param_grid, cv=5)
grid_search.fit(X_train, y_train)
print(grid_search.best_params_)
print(grid_search.best_score_)
y_pred = grid_search.predict(X_test)
print(mean_squared_error(y_test, y_pred))

{'max_depth': 6, 'n_estimators': 50}
0.8431263858254091
6.588402908612317


### 波士顿房价数据集结论

1. 首先调用决策树回归器对波士顿房价数据集进行回归分析，MSE为 $ 11.08 $
2. 然后使用XGBoost算法再次进行回归分析，MSE优化至 $ 6.56 $
3. 最后使用网格搜索调整XGB的参数，得出最优参数为：最大深度 $6$，决策树个数 $50$；最优得分为 $0.843$。在此参数下，MSE为 $ 6.59 $

In [16]:
# 四、使用决策树回归器对糖尿病数据集进行回归
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor

diabetes_df = pd.read_csv('data/diabetes.csv')
print(diabetes_df.head())

        age       sex       bmi        bp        s1        s2        s3  \
0  0.038076  0.050680  0.061696  0.021872 -0.044223 -0.034821 -0.043401   
1 -0.001882 -0.044642 -0.051474 -0.026328 -0.008449 -0.019163  0.074412   
2  0.085299  0.050680  0.044451 -0.005670 -0.045599 -0.034194 -0.032356   
3 -0.089063 -0.044642 -0.011595 -0.036656  0.012191  0.024991 -0.036038   
4  0.005383 -0.044642 -0.036385  0.021872  0.003935  0.015596  0.008142   

         s4        s5        s6  target  
0 -0.002592  0.019907 -0.017646   151.0  
1 -0.039493 -0.068332 -0.092204    75.0  
2 -0.002592  0.002861 -0.025930   141.0  
3  0.034309  0.022688 -0.009362   206.0  
4 -0.002592 -0.031988 -0.046641   135.0  


In [17]:
X, y = diabetes_df.iloc[:, :-1].values, diabetes_df.iloc[:, -1].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

tree = DecisionTreeRegressor()
tree.fit(X_train, y_train)
y_pred = tree.predict(X_test)
print(mean_squared_error(y_test, y_pred))

4902.08988764045


In [18]:
# 下面想使用随机森林对糖尿病数据集进行回归
from sklearn.ensemble import RandomForestRegressor

forest = RandomForestRegressor()
forest.fit(X_train, y_train)
y_pred = forest.predict(X_test)
print(mean_squared_error(y_test, y_pred))

3058.205806741573


In [19]:
# 使用网格搜索算法进行参数优化
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [10, 30, 50, 100],
    'max_depth': [3, 6, 9],
    'min_samples_split': [2, 4, 6, 8],
    'min_samples_leaf': [1, 2, 3, 4]
}

grid_search = GridSearchCV(RandomForestRegressor(), param_grid, cv=5)
grid_search.fit(X_train, y_train)
print(grid_search.best_params_)
print(grid_search.best_score_)
y_pred = grid_search.predict(X_test)
print(mean_squared_error(y_test, y_pred))

{'max_depth': 3, 'min_samples_leaf': 4, 'min_samples_split': 4, 'n_estimators': 50}
0.42437125391992153
2869.5343412155153


### 糖尿病数据集结论

1. 首先调用决策树回归器对糖尿病数据集进行回归分析，MSE为 $ 4902.09 $
2. 然后使用随机森林算法再次进行回归分析，MSE优化至 $ 3058.21 $
3. 最后使用网格搜索调整随机森林的参数，得出最优参数为：最大深度 $3$，分裂内部节点所需的最小样本数为 $4$，叶节点所需的最小样本数 $4$，决策树个数 $50$；最优得分为 $0.424$。在此参数下，MSE为 $ 2869.53 $